# Tugas 1 : Crawling Web Berita tempo.co

## crowling

Crawling, atau dikenal juga sebagai web crawling, adalah proses mengunjungi halaman-halaman web secara otomatis menggunakan program komputer yang disebut crawler atau spider.

Tujuan crawling adalah untuk untuk mengidentifikasi suatu laman website agar mesin pencari dapat mengelompokkannya ke kategori tertentu dengan mudah.

dalam mengambil data kita juga pasti tau tentang scraping, perbedaan antara scrapin dan crowling sendiri adalah!

Scraping, atau dikenal juga sebagai web scraping, adalah proses mengambil data spesifik dari halaman web dan mengubahnya menjadi format yang berguna (seperti CSV, Excel, atau database). Scraping melibatkan pengambilan konten tertentu dari halaman web (seperti teks, gambar, harga produk, dll.) untuk keperluan analisis, penelitian, atau aplikasi bisnis lainnya.

## berikut adalah lagkah - langkah melakukan crowling web berita

### menginstal pustaka

In [7]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd

requests digunakan untuk mendapatkan HTML dari web

BeautifulSoup digunakan untuk memparsing HTML tersebut

datetime serta timedelta digunakan untuk mengelola dan memanipulasi data waktu

### mengambil konten dari halaman web

In [8]:
all_data = []
base_url = 'https://www.tempo.co/terpopuler'
max_nasional = 50
max_bisnis = 50
count_nasional = 0
count_bisnis = 0

### Looping melalui beberapa halaman

In [9]:
for page_num in range(1, 11):  # Menambahkan lebih banyak halaman untuk memastikan target tercapai
    url = base_url if page_num == 1 else f'https://www.tempo.co/terpopuler?page={page_num}'

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Mengambil artikel dari HTML
    articles = soup.find_all('article', class_='text-card')

    # Looping melalui artikel dan ekstraksi informasi
    for article in articles:
        # Menghentikan proses jika jumlah maksimum sudah tercapai
        if count_nasional >= max_nasional and count_bisnis >= max_bisnis:
            break

        title = article.find('h2', class_='title')
        title_text = title.text.strip() if title else None

        # Mengambil link ke halaman detail berita
        link_tag = title.find('a') if title else None
        link = link_tag['href'] if link_tag and 'href' in link_tag.attrs else None

        # Mengambil tanggal berita
        date = article.find('h4', class_='date')
        if date:
            date_text = date.text.strip()
            if "jam lalu" in date_text:
                hours_ago = int(date_text.split(' ')[0])
                publish_time = datetime.now() - timedelta(hours=hours_ago)
                date_text = publish_time.strftime('%d-%m-%Y %H:%M')
        else:
            date_text = None

        # Ekstraksi kategori dari URL jika ada
        category_from_url = link.split('.')[0].replace('https://', '').replace('http://', '') if link else None
        category = category_from_url.capitalize() if category_from_url else None

        # Memeriksa kategori dan memastikan jumlah maksimum belum tercapai
        if category == 'Nasional' and count_nasional < max_nasional:
            count_nasional += 1
        elif category == 'Bisnis' and count_bisnis < max_bisnis:
            count_bisnis += 1
        else:
            continue  # Lewati jika kategori tidak sesuai atau sudah mencapai batas

        # Jika link ditemukan, lanjutkan proses scraping halaman detail berita
        if link:
            full_url = f'https://www.tempo.co{link}' if link.startswith('/') else link

            # Mendapatkan isi lengkap dari halaman detail berita
            article_response = requests.get(full_url)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # Mencoba mendapatkan isi berita dari beberapa elemen
            content_section = article_soup.find('div', class_='detail-desc') or \
                              article_soup.find('div', class_='content') or \
                              article_soup.find('article')

            full_content = None
            if content_section:
                paragraphs = content_section.find_all('p')
                full_content = '\n'.join([p.text.strip() for p in paragraphs if p.text])

            # Masukkan data ke list jika kategori cocok
            all_data.append({
                'Judul Berita': title_text,
                'Isi Berita': full_content,
                'Tanggal Berita': date_text,
                'Kategori Berita': category
            })

    # Menghentikan proses jika jumlah maksimum sudah tercapai
    if count_nasional >= max_nasional and count_bisnis >= max_bisnis:
        break

### Membuat DataFrame dari data semua halaman

In [10]:
df = pd.DataFrame(all_data)

# Mengatur ulang indeks DataFrame agar dimulai dari 1
df = df.reset_index(drop=True)
df.index += 1

### Menggunakan Styler untuk menampilkan tabel dengan border

In [11]:
styled_df = df.style.set_table_styles(
    [{'selector': 'table', 'props': [('border-collapse', 'collapse'), ('width', '100%')]},
     {'selector': 'th, td', 'props': [('border', '1px solid black'), ('padding', '8px'), ('text-align', 'left')]}]
)

display(styled_df)

### Download hasil dalam bentuk csv

In [14]:
df = pd.DataFrame.from_dict(all_data)
df
df.to_csv("data_terbaru1.csv", index=False)